In [2]:
import pandas as pd
import numpy as np

In [3]:
# Neste desafio, estamos interessados em retirar algumas estatísticas sobre a pontução de crédito dos consumidores
# separado por estado
consumidores = pd.read_csv("desafio1.csv")

In [6]:
consumidores.head()

,RowNumber,id,sobrenome,pontuacao_credito,estado_residencia,genero,idade,nivel_estabilidade,saldo_conta,numero_produtos,possui_cartao_de_credito,membro_ativo
0,1,e7f44fcbd380d4cef7e6c232cc7e37895c3fd197,6d6e0aa1b9b413e442e2fb68df14b4fc3f91de50,619,SC,F,42,2,0.00,1,1,1
1,2,28dcb083ad90512da16b9430085c2cddb8ca5e12,48e1ad846796fa314f1b4a6702b83343eb5482c5,608,RS,F,41,1,83807.86,1,0,1
2,3,774bc378f787438c9c7594e536787d07a097a54b,f2b29d5d934de615812b697132e767dea0f1e9e2,502,SC,F,42,8,159660.80,3,1,0
3,4,043a71326f7096de155e7f0c559dc62b5e4b7239,85d5c9da7cddd8109ad32a6c348fe2bb10bf99f2,699,SC,F,39,1,0.00,2,0,0
4,5,37b3bb8ca243955fb3605ec7e1c2a4607cdb3b22,ddc89c837a6933639de75f28171057060bd322df,850,RS,F,43,2,125510.82,1,1,1


In [19]:
# Como queremos apenas a pontuação de crédito e o estado, vamos retirar as informações que não nos interessam
consumidores = consumidores[['id', 'pontuacao_credito', 'estado_residencia']]

In [20]:
consumidores.describe()

,pontuacao_credito
count,7000.000000
mean,649.787000
std,97.046279
min,350.000000
25%,582.000000
50%,651.000000
75%,717.000000
max,850.000000


In [24]:
consumidores.columns

Index(['id', 'pontuacao_credito', 'estado_residencia'], dtype='object')

In [28]:
# Antes de começarmos a retirar nossas informações, vamos preparar o dataframe que irá receber elas
result = pd.DataFrame(index=['estado'],
                      columns=['moda', 'mediana', 'media', 'desvio_padrao'])
result.head()

,moda,mediana,media,desvio_padrao
estado,NaN,NaN,NaN,NaN


In [29]:
# Agora, vamos criar um dataframe apenas para os de santa catarina
sc_consumidores = consumidores.loc[consumidores['estado_residencia'] == 'SC'].copy()

In [30]:
sc_consumidores.describe()

,pontuacao_credito
count,3519.000000
mean,649.537653
std,97.233493
min,359.000000
25%,581.000000
50%,653.000000
75%,717.000000
max,850.000000


In [34]:
# Vamos retirar a média
sc_mean = sc_consumidores['pontuacao_credito'].mean()
sc_mean

649.5376527422563

In [36]:
# Agora a mediana
sc_median = sc_consumidores['pontuacao_credito'].median()
sc_median

653.0

In [38]:
# Moda
sc_mode = sc_consumidores['pontuacao_credito'].mode()
sc_mode

0    850
dtype: int64

In [41]:
# E agora, o desvio padrão
sc_std = sc_consumidores['pontuacao_credito'].std()
sc_std

97.233492793433

In [42]:
# Podemos observar que todas essas informações já vieram dentro do comando sc_consumidores.describe()
# Será que conseguimos realizar essa operação sem ter que fazer manualmente para cada um dos dataframes?

consumidores.head()

,id,pontuacao_credito,estado_residencia
0,e7f44fcbd380d4cef7e6c232cc7e37895c3fd197,619,SC
1,28dcb083ad90512da16b9430085c2cddb8ca5e12,608,RS
2,774bc378f787438c9c7594e536787d07a097a54b,502,SC
3,043a71326f7096de155e7f0c559dc62b5e4b7239,699,SC
4,37b3bb8ca243955fb3605ec7e1c2a4607cdb3b22,850,RS


In [43]:
consumidores.groupby('estado_residencia').describe()

pontuacao_credito                                       \
                              count        mean        std    min    25%   
estado_residencia                                                          
PR                           1731.0  648.961294  98.607186  350.0  581.0   
RS                           1750.0  651.105143  95.136598  350.0  585.0   
SC                           3519.0  649.537653  97.233493  359.0  581.0   

                                        
                     50%    75%    max  
estado_residencia                       
PR                 650.0  719.5  850.0  
RS                 650.0  717.0  850.0  
SC                 653.0  717.0  850.0

In [76]:
# Dessa forma, podemos criar um novo dataframe pegando apenas as colunas que nos interessam 
result_df = consumidores.groupby('estado_residencia')['pontuacao_credito'].describe()
result_df = result_df[['mean', 'std', '50%', 'max']]
result_df

,mean,std,50%,max
estado_residencia,,,,
PR,648.961294,98.607186,650.0,850.0
RS,651.105143,95.136598,650.0,850.0
SC,649.537653,97.233493,653.0,850.0


In [78]:
# Agora, vamos renomear as colunas 
result_df.rename(columns={'mean': 'media', '50%': 'mediana', 'max': 'moda', 'std': 'desvio_padrao'}, inplace=True)
result_df.head()

,media,desvio_padrao,mediana,moda
estado_residencia,,,,
PR,648.961294,98.607186,650.0,850.0
RS,651.105143,95.136598,650.0,850.0
SC,649.537653,97.233493,653.0,850.0


In [81]:
# Agora é só exportar nosso resultado para json, conforme esperado pelo desafio
result_df.to_json('submission.json', orient='index')